In [1]:
import os

In [5]:
os.getenv("user"), os.getenv("warehouse"), os.getenv("schema"), os.getenv("database"), os.getenv("role"), os.getenv("account"), os.getenv("password")

('AKSHAYA',
 'FOSFOR_SOLUTIONS_WH',
 'TTH_REV_OPT_Schema',
 'TTH_DB',
 'AKSHAYA',
 'ug94937.us-east4.gcp',
 'Password@2023')